In [156]:
import json
import numpy as np

In [157]:
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, Dense, InputLayer, Concatenate, Dropout

In [158]:
with open("../data/json_dataset/annotations_train.json") as fp:
    train = json.load(fp)

In [159]:
with open("../data/json_dataset/annotations_test.json") as fp:
    test = json.load(fp)

In [160]:
def build_model(layers=3, units=200):
    object_input = Input((100,))
    subject_input = Input((100,))
    concat = Concatenate()([object_input, subject_input])
    inputs = concat

    for _ in range(LAYERS):
        inputs = Dense(units=UNITS, activation='relu')(inputs)
        inputs = Dropout(0.5)(inputs)

    relation = Dense(units=70, activation="softmax")(inputs)

    model = Model(inputs=(object_input, subject_input), outputs=relation)
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    return model

In [161]:
model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
concatenate_16 (Concatenate)    (None, 200)          0           input_31[0][0]                   
                                                                 input_32[0][0]                   
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 200)          40200       concatenate_16[0][0]             
__________

In [166]:
def get_data(src, train=True):
    objects = []
    subjects = []
    relations = []

    for anns in src.values():
        for ann in anns:
            objects.append(ann['object']['category'])
            subjects.append(ann['subject']['category'])
            relations.append(ann['predicate'])
            
    xobj = to_categorical(objects)
    xsub = to_categorical(subjects)
    if train:
        y = to_categorical(relations)
    else:
        y = np.asarray(relations)
    
    return (xobj, xsub), y

In [167]:
x,y = get_data(train)
model.fit(x=x, y=y, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

Train on 27319 samples, validate on 3036 samples
Epoch 1/100
27319/27319 [==============================] - 4s 145us/step - loss: 1.6393 - val_loss: 1.6182
Epoch 2/100
27319/27319 [==============================] - 4s 147us/step - loss: 1.6284 - val_loss: 1.6076
Epoch 3/100
27319/27319 [==============================] - 4s 156us/step - loss: 1.6258 - val_loss: 1.6122
Epoch 4/100
27319/27319 [==============================] - 4s 149us/step - loss: 1.6171 - val_loss: 1.6009
Epoch 5/100
27319/27319 [==============================] - 4s 148us/step - loss: 1.6070 - val_loss: 1.6071
Epoch 6/100
27319/27319 [==============================] - 4s 149us/step - loss: 1.6046 - val_loss: 1.6052


In [170]:
x,y = get_data(test, train=False)
ypred = model.predict(x)
ypred = ypred.argmax(axis=-1)
(y == ypred).astype(float).mean()

0.5331238544121498